<h1 style="text-align: center;">SentencePiece Experiments</h1>

# Imports and Data
**Note:** Run the file in google colab if you wnat to train the model on your own. Model is too big to be trained locally.

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import sentencepiece as spm
import pandas as pd
import os

In [4]:
df = pd.read_json("/content/drive/MyDrive/Colab Notebooks/wiki_arm_newest.json", orient ='columns', compression = 'infer')
df

,id,url,title,text
0,800,https://hy.wikipedia.org/wiki/%D5%80%D5%A1%D5%...,Հայաստան,Հայաստանը դեպի ծով ելք չունեցող պետություն է Հ...
1,1027,https://hy.wikipedia.org/wiki/%D4%BC%D5%A5%D5%...,Լեռնային Ղարաբաղ,"Արցախ (Լեռնային Ղարաբաղ), պատմական տարածաշրջան..."
2,1032,https://hy.wikipedia.org/wiki/%D4%B7%D5%B5%D5%...,Էյսիդ ջազ,"Էյսիդ ջազ (, անգլերեն acid` քիմիական թթու և ja..."
3,1033,https://hy.wikipedia.org/wiki/%D5%80%D5%A1%D5%...,Հայկական խոհանոց,"Հայկական խոհանոց, ունի մի շարք առանձնահատկությ..."
4,1042,https://hy.wikipedia.org/wiki/%D4%B3%D6%80%D5%...,Գրաֆիտի,Գրաֆիտի („գրաֆիտո“ լատիներեն նշանակում է «քերթ...
...,...,...,...,...
307811,1244257,https://hy.wikipedia.org/wiki/%D5%85%D5%B8%D6%...,Յորգոս Կարագունիս,"Յորգոս Կարագկունիս (, ), հայտնի է որպես Յորգոս..."
307812,1244261,https://hy.wikipedia.org/wiki/%D4%BF%D5%AB%D5%...,Կիտա Չխենկելի,"Կիտա ""Պետրե"" Չխենկելի (, արևմտյան գրականության..."
307813,1244277,https://hy.wikipedia.org/wiki/%D5%84%D5%A5%D5%...,Մեշգինշահր,"Մեշգինշահր (), քաղաք Իրանի հյուսիս-արևմուտքում..."
307814,1244298,https://hy.wikipedia.org/wiki/%D5%8A%D5%A1%D5%...,Պաուլ (ութոտնուկ),"Պաուլ ութոտնուկ (, 2008 թվականի հունվար, Ուեյմ..."


In [5]:
corpus = ' '.join(df['text'].tolist())

In [6]:
with open('corpus.txt', 'w') as file:
    file.write(corpus)

file.close

<function TextIOWrapper.close()>

# Training SentencePiece BPE Tokenizer

In [3]:
# train a sentencepiece model on it
# the settings here are (best effort) those used for training Llama 2
import os

options = dict(
  # input spec
  input="./corpus.txt",
  input_format="text",
  # output spec
  model_prefix="armenian_sentencepiece_tokenizer", # output filename prefix
  # algorithm spec
  # BPE alg
  model_type="bpe",
  vocab_size=30000,
  # normalization
  normalization_rule_name="identity",
  remove_extra_whitespaces=False,
  input_sentence_size=200000000, # max number of training sentences
  max_sentence_length=4192, # max number of bytes per sentence
  seed_sentencepiece_size=1000000,
  shuffle_input_sentence=True,
  # rare word treatment
  character_coverage=0.99995,
  byte_fallback=False,
  # merge rules
  split_digits=True,
  split_by_unicode_script=True,
  split_by_whitespace=True,
  split_by_number=True,
  max_sentencepiece_length=16,
  add_dummy_prefix=True,
  allow_whitespace_only_pieces=True,
  # special tokens
  unk_id=0, # the UNK token MUST exist
  bos_id=1,
  eos_id=2,
  pad_id=-1,
  # systems
  num_threads=os.cpu_count(), # use ~all system resources
)

spm.SentencePieceTrainer.train(**options)


In [4]:
sp = spm.SentencePieceProcessor()
sp.load('./armenian_sentencepiece_tokenizer.model')
vocab = [[sp.id_to_piece(idx), idx] for idx in range(sp.get_piece_size())]
vocab

[['<unk>', 0],
 ['<s>', 1],
 ['</s>', 2],
 ['ու', 3],
 ['ան', 4],
 ['եր', 5],
 ['ար', 6],
 ['ակ', 7],
 ['ին', 8],
 ['այ', 9],
 ['ում', 10],
 ['▁հ', 11],
 ['ներ', 12],
 ['որ', 13],
 ['▁է', 14],
 ['ել', 15],
 ['ութ', 16],
 ['ական', 17],
 ['ուն', 18],
 ['ությ', 19],
 ['▁մ', 20],
 ['ամ', 21],
 ['▁կ', 22],
 ['են', 23],
 ['ատ', 24],
 ['ավ', 25],
 ['աս', 26],
 ['▁▁', 27],
 ['ություն', 28],
 ['ագ', 29],
 ['ով', 30],
 ['▁թ', 31],
 ['▁գ', 32],
 ['▁ն', 33],
 ['▁և', 34],
 ['▁տ', 35],
 ['▁Ա', 36],
 ['ետ', 37],
 ['աց', 38],
 ['▁|', 39],
 ['▁ա', 40],
 ['▁(', 41],
 ['ած', 42],
 ['▁դ', 43],
 ['ոն', 44],
 ['ալ', 45],
 ['իր', 46],
 ['▁բ', 47],
 ['ստ', 48],
 ['ների', 49],
 ['ող', 50],
 ['▁պ', 51],
 ['▁որ', 52],
 ['անի', 53],
 ['ես', 54],
 ['եղ', 55],
 ['ության', 56],
 ['▁թվ', 57],
 ['ահ', 58],
 ['▁վ', 59],
 ['ային', 60],
 ['ից', 61],
 ['▁թվական', 62],
 ['▁ս', 63],
 ['ազ', 64],
 ['այի', 65],
 ['▁Հ', 66],
 ['րա', 67],
 ['▁են', 68],
 ['▁||', 69],
 ['աղ', 70],
 ['առ', 71],
 ['▁Մ', 72],
 ['▁Ս', 73],
 ['երի', 7

In [5]:
sp = spm.SentencePieceProcessor()
sp.load('armenian_sentencepiece_tokenizer.model')
vocab = [[sp.id_to_piece(idx), idx] for idx in range(sp.get_piece_size())]
vocab

[['<unk>', 0],
 ['<s>', 1],
 ['</s>', 2],
 ['ու', 3],
 ['ան', 4],
 ['եր', 5],
 ['ար', 6],
 ['ակ', 7],
 ['ին', 8],
 ['այ', 9],
 ['ում', 10],
 ['▁հ', 11],
 ['ներ', 12],
 ['որ', 13],
 ['▁է', 14],
 ['ել', 15],
 ['ութ', 16],
 ['ական', 17],
 ['ուն', 18],
 ['ությ', 19],
 ['▁մ', 20],
 ['ամ', 21],
 ['▁կ', 22],
 ['են', 23],
 ['ատ', 24],
 ['ավ', 25],
 ['աս', 26],
 ['▁▁', 27],
 ['ություն', 28],
 ['ագ', 29],
 ['ով', 30],
 ['▁թ', 31],
 ['▁գ', 32],
 ['▁ն', 33],
 ['▁և', 34],
 ['▁տ', 35],
 ['▁Ա', 36],
 ['ետ', 37],
 ['աց', 38],
 ['▁|', 39],
 ['▁ա', 40],
 ['▁(', 41],
 ['ած', 42],
 ['▁դ', 43],
 ['ոն', 44],
 ['ալ', 45],
 ['իր', 46],
 ['▁բ', 47],
 ['ստ', 48],
 ['ների', 49],
 ['ող', 50],
 ['▁պ', 51],
 ['▁որ', 52],
 ['անի', 53],
 ['ես', 54],
 ['եղ', 55],
 ['ության', 56],
 ['▁թվ', 57],
 ['ահ', 58],
 ['▁վ', 59],
 ['ային', 60],
 ['ից', 61],
 ['▁թվական', 62],
 ['▁ս', 63],
 ['ազ', 64],
 ['այի', 65],
 ['▁Հ', 66],
 ['րա', 67],
 ['▁են', 68],
 ['▁||', 69],
 ['աղ', 70],
 ['առ', 71],
 ['▁Մ', 72],
 ['▁Ս', 73],
 ['երի', 7

In [6]:
encoded_ids = sp.encode("բարև ոնց ե՞ս՝ կոֆե խմե՞նք ")
print(encoded_ids)

[998, 28995, 28962, 44, 28980, 1263, 23231, 29019, 22, 1230, 28968, 1411, 27554, 28964, 28985, 28962]


In [7]:
encoded_text = [sp.id_to_piece(idx) for idx in encoded_ids]
encoded_text

['▁բար',
 'և',
 '▁',
 'ոն',
 'ց',
 '▁ե',
 '՞ս',
 '՝',
 '▁կ',
 'ոֆ',
 'ե',
 '▁խմ',
 'ե՞',
 'ն',
 'ք',
 '▁']

In [8]:
decoded_text = sp.decode(encoded_text).replace('▁', ' ')
decoded_text

'բարև ոնց ե՞ս՝ կոֆե խմե՞նք '